In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/amrzr/Work/Codes/Offline_IMOEA_Framework/')
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from scipy.stats import qmc

import copy
import pickle
import os

In [14]:
data_folder = '/home/amrzr/Work/Codes/data'
init_folder = data_folder + '/constraint_handling_dataset'
file_instances = init_folder + '/test_instances.csv'
data_instances = pd.read_csv(file_instances)
problem_testbench = 'DTLZ'
all_problems = data_instances["problem"].values
all_n_vars = data_instances["nvars"].values
all_objs = data_instances["K"].values
all_sample_size = data_instances["N"].values
all_ndim = data_instances["ndim"].values
all_bound_valL = data_instances["bound_valL"].values
all_bound_valU = data_instances["bound_valU"].values
size_instance = np.size(all_problems)
boundoldL= 0 # for DTLZ
boundoldU = 1
nruns = 35
path = data_folder+'/constraint_handling_dataset/' + problem_testbench 

In [8]:
"""
# problem parameters
problem_name = 'DTLZ2'
nobjs = 2
nvars = 5
nruns = 35
N = 250
bound_valL=0.2
bound_valU=0.9

boundoldL= 0 # for DTLZ
boundoldU = 1  
ndim = nvars # nvars...for box constraints, 1,2...for dimensionwise constraints

# end of parameters
"""

In [9]:

"""data_folder = '/home/amrzr/Work/Codes/data'
path = data_folder+'/constraint_handling_dataset/' + problem_testbench

problem_spec ='tests_' + problem_name +'_'+ str(N) + '_' + str(nobjs) + '_' + \
    str(nvars) +  '_b'+str(ndim) +'_' + str(bound_valL).replace('.','') + \
        str(bound_valU).replace('.','')
 """

In [16]:
for instance in range(size_instance):
    prob = all_problems[instance]
    n_vars = all_n_vars[instance]
    obj = all_objs[instance]
    sample_size = all_sample_size[instance]
    ndim = all_ndim[instance]
    bound_valL = all_bound_valL[instance]
    bound_valU = all_bound_valU[instance]

    problem_spec = prob +'_'+ str(sample_size) + '_' + str(obj) + '_' + \
        str(n_vars) +  '_b'+str(ndim) +'_' + str(bound_valL).replace('.','') + \
            str(bound_valU).replace('.','')

    boundvecL = bound_valL* np.ones(n_vars)
    boundvecU = bound_valU* np.ones(n_vars)
    boundvecL = boundoldL* np.ones(n_vars)
    boundvecU = boundoldU* np.ones(n_vars)
    boundvecL[:ndim]=np.ones(ndim)*bound_valL
    boundvecU[:ndim]=np.ones(ndim)*bound_valU
    boundL = np.tile(boundvecL,(sample_size,1))
    boundU = np.tile(boundvecU,(sample_size,1))


    for run in range(nruns):
        sampler = qmc.LatinHypercube(d=n_vars)
        data = sampler.random(n=sample_size)*(boundoldU-(boundoldL)) + (boundoldL)
        # print(data)

        testproblem = test_problem_builder(prob, n_of_objectives= obj, n_of_variables=n_vars)
        obj_val = testproblem.evaluate(data)


        failed_loc = np.where(np.all(data >= boundL, axis=1) & np.all(data <= boundU, axis=1))
        stat_success = np.ones((sample_size,1))
        stat_success[failed_loc,0]=0

        data_failed=data[failed_loc[0],:]
        #ax.scatter(data_failed[:,0],data_failed[:,1],data_failed[:,2], c='black', marker=',')
        obj_vals = obj_val[0]
        obj_success = obj_vals[np.where(stat_success==1)[0],:]

        #np.size(failed_loc)
        #print(np.size(failed_loc))

        data_class = pd.DataFrame(np.hstack((data, stat_success)))
        data_success = pd.DataFrame(np.hstack((data[np.where(stat_success==1)[0],:],obj_success)))
        data_all = pd.DataFrame(np.hstack((np.hstack((data, obj_vals)),stat_success)))

        data_class.to_csv(path + '/' + problem_spec + '_'+ str(run)+ '_data_class.csv',index=False)
        data_success.to_csv(path + '/' + problem_spec + '_'+ str(run) + '_data_success.csv',index=False)
        data_all.to_csv(path + '/' + problem_spec  + '_'+ str(run) + '_data_all.csv',index=False)